In [ ]:
# qubit reuse real hardware
# Classical register is renamed while using qubit reuse plugin
# qubit reuse plugin is needed to be installed: 
#   Go to       https://github.com/qiskit-community/qiskit-qubit-reuse/blob/main/README.md      for installation instruction 

In [10]:
# necessary libraries: Circuit Construction, transpilation, For simulation, visualization, JSON
# Circuit Construction
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

#Transpilation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# For Simulation
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
aer_sim = AerSimulator()                              # for ideal simulation
service = QiskitRuntimeService()
backend_name = "ibm_sherbrooke"
backend = service.get_backend(backend_name)            # for real hardware


from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler_ideal = Sampler(backend=aer_sim)
sampler_noisy = Sampler(backend=backend)
options = sampler_noisy.options
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = 'XY4'
#Visualization
from qiskit.visualization import plot_histogram



#Post-processing, Fidelity and JSON
import json
import matplotlib.pyplot as plt
import statistics

from qiskit.quantum_info.analysis import hellinger_fidelity

In [11]:
# function for Circuit Construction (GHZ state for how many number of qubits, that is num_qubit argument of the function)

def get_ghz_circ_static(num_qubit):
    if num_qubit%2==1:                                                     #odd no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-1)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-1):
            qc.cx(i,i+1)
            qc.cx(num_qubit-i-1,num_qubit-i-2)
        

    else:                                                                    #even no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-2)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-2):
            qc.cx(i,i+1)
            qc.cx(num_qubit-1-i-1,num_qubit-i-3)
        qc.cx(num_qubit-2,num_qubit-1)
    for i in range(num_qubit):
        qc.measure(qr[i],cr2[i])
    return qc

qc=get_ghz_circ_static(6)

In [12]:
# Transpilation ideal
pm_i = generate_preset_pass_manager(backend=aer_sim, optimization_level=3,init_method="qubit_reuse")
qc_ideal = pm_i.run(qc)

# Transpilation noisy
pm_r = generate_preset_pass_manager(backend=backend, optimization_level=3,init_method="qubit_reuse")
qc_noisy = pm_r.run(qc)

In [13]:
#Run the Circuit
result_ideal = sampler_ideal.run([qc_ideal]).result()


In [7]:
# run experiment on real hardware
job = sampler_noisy.run([qc_noisy])
print(job.job_id())

cwhr0ch9r49g008krx0g


In [14]:
#Post-processing 
job_id = 'cwhr0ch9r49g008krx0g' 
job = service.job(job_id)
result_noisy = job.result()

In [21]:
result_noisy

PrimitiveResult([{'__type__': 'SamplerPubResult', '__value__': {'data': DataBin(c6=BitArray(<shape=(), num_shots=4096, num_bits=6>)), 'metadata': {'circuit_metadata': {}}}}], metadata={'execution': {'execution_spans': {'__type__': 'ExecutionSpanCollection', '__value__': {'spans': [{'__type__': 'ExecutionSpan', '__value__': {'start': datetime.datetime(2024, 10, 31, 18, 9, 21, 444458, tzinfo=tzutc()), 'stop': datetime.datetime(2024, 10, 31, 18, 9, 50, 159380, tzinfo=tzutc()), 'data_slices': {'0': [[4096], 0, 4096]}}}]}}}, 'version': 2})

In [17]:
qc_ideal.cregs

[ClassicalRegister(6, 'c8')]

In [26]:
# Get Hellinger Fidelity
counts_ideal = result_ideal[0].data.c8.get_counts()
counts_noisy = result_noisy[0]['__value__']['data'].c6.get_counts()
#counts_noisy = result_noisy[0].data.c10.get_counts()
Fidelity_qubit_reuse_real = hellinger_fidelity(counts_ideal, counts_noisy)

In [27]:
Fidelity_qubit_reuse_real

0.4489302696456745

In [28]:
#Store Result into JSON file
import json
with open("Fidelity_scripts_comparison.json", "r") as jsonfile:
    data1 = json.load(jsonfile)

new_data1 =  {
    "Fidelity_qubit_reuse_real" : Fidelity_qubit_reuse_real,    
    }

    

data1.update(new_data1)

with open("Fidelity_scripts_comparison.json", "w") as jsonfile:
    json.dump(data1, jsonfile)